In [ ]:
!python3 'mnist_federated.py' --ExperimentName="MNIST-Federated-R20-E5-B64-DA" \
                              --GroupName="MNIST-Federated-R20-E5-B64-DA" \
                              --NumberOfClients=23 \
                              --FractionOfClients=1 \
                              --NumberOfRounds=20 \
                              --BatchSize=64 \
                              --NumberOfEpochs=5 \
                              --DataAugmentation="True"

### Apply test set on best selected model "Centralized eval"

In [ ]:
from tensorflow import keras
from keras.datasets import mnist
import numpy
import sys

sys.path.append('../..')
import experiments.mnistCentralized.mnist_centralized as Experiment


model_name= "MNIST-Federated-R100-E1-B64/R-97"
model_from_disk = keras.models.load_model(f"models/{model_name}")

_, _, _, _, (test_x, test_y) = Experiment.get_data(64, "False")


result = model_from_disk.evaluate(x=test_x, y=test_y , batch_size=64)

print(f"Experiment model: {model_name}")
print(f"loss={result[0]}\n tp={result[1]}\n fp={result[2]}\n tn={result[3]}\n fn={result[4]}\n accuracy={result[5]}\n presision={result[6]}\n recall={result[7]}\n auc={result[8]}\n prc={result[9]}")

### Apply test set on best model "Federated eval"

In [2]:
from tensorflow import keras
from keras.datasets import mnist
import numpy as np
import sys

sys.path.append('../..')
import utils.dataloaders.mnist_dataloader as DataLoader


#model_name= "../mnistCentralized/models/MNIST-Centralized-B64-E100/Epoch-88"
model_name= "../mnistFederated/models/MNIST-Federated-R100-E1-B64/R-97"

model_from_disk = keras.models.load_model(f"{model_name}")

all_results = []
batch_size = 64
number_of_clients=23
for client_id in range(23):
    base_full_path = f"/mnt/data/FederatedLearning/data/MNIST/partitions/Client-{client_id+1}"

    test_loader = DataLoader.MNISTDataloader(base_full_path, batch_size, dataset_split="Test", data_augmentation = "False")
    result = model_from_disk.evaluate(test_loader)
    all_results.append(result)


# acum all 23 clients metrics
aggregated_results = [0,0,0,0,0,0,0,0,0,0]    
for partial_results in all_results:
    for index, metric in enumerate(partial_results):
        aggregated_results[index] += metric
        
        
# take mean        
for index, agg_res in enumerate(aggregated_results):
    aggregated_results[index] = aggregated_results[index]/number_of_clients
    
     
    
standart_deviation = []    
# collect std    
for i in range(10):
    metrics_groups = [item[i] for item in all_results]
    standart_deviation.append(np.std(metrics_groups))

print("std")
print(standart_deviation)
    

metric_names =  ['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'presision', 'recall', 'auc', 'prc']
for i, final_res in enumerate(aggregated_results):
    print(f"{metric_names[i]} {final_res} +/- {standart_deviation[i]}")
    

#print(f"Experiment model: {model_name}")
#print(f"loss={result[0]}\n tp={result[1]}\n fp={result[2]}\n tn={result[3]}\n fn={result[4]}\n accuracy={result[5]}\n presision={result[6]}\n recall={result[7]}\n auc={result[8]}\n prc={result[9]}")

6/6 [==============================] - 0s 9ms/step - loss: 0.0055 - tp: 384.0000 - fp: 0.0000e+00 - tn: 3456.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - prc: 1.0000
std
[0.015210580051037762, 1.4208812729015818, 1.1970347737335831, 1.197034773733583, 1.4208812729015818, 0.003573310360565877, 0.003120195386580387, 0.003700213789280379, 0.0007997143599231078, 0.002305993159787896]
loss 0.02222469909136872 +/- 0.015210580051037762
tp 381.7391304347826 +/- 1.4208812729015818
fp 1.9565217391304348 +/- 1.1970347737335831
tn 3454.0434782608695 +/- 1.197034773733583
fn 2.260869565217391 +/- 1.4208812729015818
accuracy 0.9943387715712838 +/- 0.003573310360565877
presision 0.9948998663736426 +/- 0.003120195386580387
recall 0.9941123205682506 +/- 0.003700213789280379
auc 0.9994849847710651 +/- 0.0007997143599231078
prc 0.9983816613321719 +/- 0.002305993159787896


In [ ]:
import sys
sys.path.append('../..')

import client.mnist.mnist_client as Client

client = Client.MNISTClient(0, 3, 4, "bla", "bla", 128)
#testing = client.test_generator

training = client.training_generator

X, y = training.__getitem__(i)

    
print(y)

In [ ]:
from tensorflow import keras
from keras.datasets import mnist

model = keras.models.load_model('models/model-MNIST-Federated-test-2022-05-11')
(train_X, train_y), (test_X, test_y) = mnist.load_data()

result = model.evaluate(test_X, y=keras.utils.to_categorical(test_y,10))



In [ ]:
import numpy


result_maxed = numpy.argmax(result, axis=1)

correct = 0
wrong = 0

for ground_index in range(len(test_y)):
    if test_y[ground_index] == result_maxed[ground_index]:
        correct += 1
    else:
        wrong += 1
        

print(f"Correct {correct}")
print(f"Wrong {wrong}")
print(f"total {correct + wrong }")
print(f"accuracy {correct /(correct + wrong) }")



    